---

**Welcome to the Magic of Words! - Text-to-Image with Stable Diffusion**

Welcome! Whether you're a writer, designer, developer, or someone curious about the intersection of text and visuals, this guide is for you.

Have you ever wanted to convert a description or phrase into an image? With Text-to-Image Stable Diffusion, you can do just that. Input a description like "a quiet sunset over the ocean" or "a cityscape at night," and see it translated into a visual representation.

This process is powered by Intel's Max Series GPUs. These GPUs are optimized for tasks like Text-to-Image Stable Diffusion, ensuring efficient and accurate conversions.

This guide will walk you through examples, help you understand the outputs, and give pointers on how to get the best results. No technical deep dives, just clear steps and explanations.

Ready to turn your words into visuals? Let's get started by setting up few things and importing the required python packages!

In [23]:
# install and import required dependencies
# %pip install transformers accelerate diffusers pillow >/dev/nul
import os
import random
import time
import warnings

# Suppress warnings for a cleaner output.
warnings.filterwarnings("ignore")

import requests
import torch
import intel_extension_for_pytorch as ipex  # Used for optimizing PyTorch models
from PIL import Image

from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from io import BytesIO
import torch.nn as nn
import time
from typing import List, Dict, Tuple

---

**A Glimpse Behind the Scenes**

For those intrigued by the underpinnings of this adventure, let's delve into the technicalities of the code. No worries if you're not aiming for a deep dive; understanding this isn't a prerequisite to harness the magic of the notebook. But for the tech-curious, let's dissect:

- **Class Blueprint**: At the heart of our operations is the `Text2ImgModel` class. This acts as the central hub, choreographing the process of transforming textual prompts into visual masterpieces.

- **Initialization**: When we instantiate this class, we have the liberty to tailor its behavior. This includes selecting the model's path, the device (defaulting to Intel's "xpu"), the data type, and more. There are also provisions to supercharge the model's performance via optimizations.

- **Model Onboarding**: The `_load_pipeline` function is where we bring the pre-trained model onboard. This sets the stage for all our text-to-image transformations.

- **Optimization**: Performance is paramount. With the `_optimize_pipeline` and `optimize_pipeline` methods, we leverage Intel-specific optimizations using the Intel Extension For PyTorch (IPEX) to ensure our model runs like the wind.

- **Warming Up**: Just like a car on a cold morning, our model benefits from a brief warm-up. The `warmup_model` method handles this, ensuring that our model is primed and ready for the tasks ahead.

- **The Grand Finale - Image Generation**: The `generate_images` method is where dreams meet reality. It interprets the textual prompts, consults with the model, and then crafts images that encapsulate the essence of the prompts. You can specify the number of images, the number of noise removal steps, and even the save path for these artworks.

Intrigued? Dive into the code below and see how we've meticulously tailored every aspect, ensuring it's primed for Intel GPUs.

---


In [17]:
class Text2ImgModel:
    """
    Text2ImgModel is a class for generating images based on text prompts using a pretrained model.

    Attributes:
    - device: The device to run the model on. Default to "xpu" - Intel dGPUs.
    - pipeline: The loaded model pipeline.
    - data_type: The data type to use in the model.
    """

    def __init__(
        self,
        model_id_or_path: str,
        device: str = "xpu",
        torch_dtype: torch.dtype = torch.bfloat16,
        optimize: bool = True,
        enable_scheduler: bool = False,
        warmup: bool = True
    ) -> None:
        """
        The initializer for Text2ImgModel class.

        Parameters:
        - model_id_or_path: The identifier or path of the pretrained model.
        - device: The device to run the model on. Default is "xpu".
        - torch_dtype: The data type to use in the model. Default is torch.bfloat16.
        - optimize: Whether to optimize the model after loading. Default is True.
        """

        self.device = device
        self.pipeline = self._load_pipeline(
            model_id_or_path, torch_dtype, enable_scheduler
        )
        self.data_type = torch_dtype
        if optimize:
            start_time = time.time()
            print("Optimizing the model...")
            self.optimize_pipeline()
            print(
                "Optimization completed in {:.2f} seconds.".format(
                    time.time() - start_time
                )
            )
        if warmup:
            self.warmup_model()

    def _load_pipeline(
        self,
        model_id_or_path: str,
        torch_dtype: torch.dtype,
        enable_scheduler: bool
    ) -> DiffusionPipeline:
        """
        Loads the pretrained model and prepares it for inference.

        Parameters:
        - model_id_or_path: The identifier or path of the pretrained model.
        - torch_dtype: The data type to use in the model.

        Returns:
        - pipeline: The loaded model pipeline.
        """

        print("Loading the model...")
        pipeline = DiffusionPipeline.from_pretrained(
            model_id_or_path,
            torch_dtype=torch_dtype,
            use_safetensors=True,
            variant="fp16",
        )
        if enable_scheduler:
            pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
                pipeline.scheduler.config
            )
        pipeline = pipeline.to(self.device)
        print("Model loaded.")
        return pipeline

    def _optimize_pipeline(self, pipeline: DiffusionPipeline) -> DiffusionPipeline:
        """
        Optimizes the model for inference using ipex.

        Parameters:
        - pipeline: The model pipeline to be optimized.

        Returns:
        - pipeline: The optimized model pipeline.
        """

        for attr in dir(pipeline):
            if isinstance(getattr(pipeline, attr), nn.Module):
                setattr(
                    pipeline,
                    attr,
                    ipex.optimize(
                        getattr(pipeline, attr).eval(),
                        dtype=pipeline.text_encoder.dtype,
                        inplace=True,
                    ),
                )
        return pipeline

    def warmup_model(self):
        """
        Warms up the model by generating a sample image.
        """
        print("Setting up model...")
        start_time = time.time()
        self.generate_images(
            prompt="A beautiful sunset over the mountains",
            num_images=1,
            save_path="/tmp",
        )
        print(
            "Model is set up and ready! Warm-up completed in {:.2f} seconds.".format(
                time.time() - start_time
            )
        )

    def optimize_pipeline(self) -> None:
        """
        Optimizes the current model pipeline.
        """

        self.pipeline = self._optimize_pipeline(self.pipeline)

    def generate_images(
        self,
        prompt: str,
        num_inference_steps: int = 50,
        num_images: int = 5,
        save_path: str = "output",
    ) -> List[Image.Image]:
        """
        Generates images based on the given prompt and saves them to disk.

        Parameters:
        - prompt: The text prompt to generate images from.
        - num_inference_steps: Number of noise removal steps.
        - num_images: The number of images to generate. Default is 5.
        - save_path: The directory to save the generated images in. Default is "output".

        Returns:
        - images: A list of the generated images.
        """

        images = []
        for i in range(num_images):
            with torch.xpu.amp.autocast(
                enabled=True if self.data_type != torch.float32 else False,
                dtype=self.data_type,
            ):
                image = self.pipeline(
                    prompt=prompt,
                    num_inference_steps=num_inference_steps,
                    #negative_prompt=negative_prompt,
                ).images[0]
                if not os.path.exists(save_path):
                    try:
                        os.makedirs(save_path)
                    except OSError as e:
                        print("Failed to create directory", save_path, "due to", str(e))
                        raise
            output_image_path = os.path.join(
                save_path,
                f"{'_'.join(prompt.split()[:3])}_{i}.png",
            )
            image.save(output_image_path)
            images.append(image)
        return images


---

**Setting Up the Text-to-Image Interface**

The following section is dedicated to creating an intuitive interface right within this notebook, providing a seamless experience to translate your textual prompts into captivating visuals.

- **Model Selection**: Pick your desired pre-trained model from the available list.
- **Prompt Input**: Enter your creative textual prompt, the muse for the image to be generated.
- **Number of Images**: Specify the number of visual interpretations you'd like to see for your prompt.
- **Enhancement**: Toggle an option to sprinkle some automatic enhancements to your prompt, potentially enriching the outcome.

After setting your preferences, all it takes is a button click to witness the text's metamorphosis into imagery!

Let's lay the groundwork for this experience:

---


In [25]:
import os
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mp_img
import validators

from IPython.display import clear_output
from IPython.display import Image as IPImage
import ipywidgets as widgets

def prompt_to_image():
    out = widgets.Output()

    output_dir = "output"
    model_ids = [
        "CompVis/stable-diffusion-v1-4",
        "stabilityai/stable-diffusion-2-1",
        "stabilityai/stable-diffusion-xl-base-1.0",
    ]    
    
    model_dropdown = widgets.Dropdown(
        options=model_ids,
        value=model_ids[0],
        description="Select Model:",
    )    
    prompt_text = widgets.Text(
        value="",
        placeholder="Enter your prompt",
        description="Prompt:",
    )    
    num_images_slider = widgets.IntSlider(
        value=2,
        min=1,
        max=10,
        step=1,
        description="Num of Images:",
    )    
    enhance_checkbox = widgets.Checkbox(
        value=False,
        description="Auto enhance the prompt?",
        disabled=False,
        indent=False
    )
    
    layout = widgets.Layout(margin="0px 50px 10px 0px")
    button = widgets.Button(description="Generate Images!")   
    left_box = VBox([model_dropdown, prompt_text, num_images_slider], layout=layout)
    right_box = VBox([enhance_checkbox], layout=layout)
    user_input_widgets = HBox([left_box, right_box], layout=layout)  
    display(user_input_widgets)
    display(button)
    display(out)

    
    def on_submit(button):
        with out:
            clear_output(wait=True)
            print("\nGenerating Images, once generated images will be saved `./output` dir, please wait...")
            selected_model_index = model_ids.index(model_dropdown.value)
            model_id = model_ids[selected_model_index]
            prompt = prompt_text.value
            num_images = num_images_slider.value
            model = Text2ImgModel(model_id, device="xpu")
            
            enhancements = [
                "dark",
                "purple light",
                "dreaming",
                "cyberpunk",
                "ancient" ", rustic",
                "gothic",
                "historical",
                "punchy",
                "photo" "vivid colors",
                "4k",
                "bright",
                "exquisite",
                "painting",
                "art",
                "fantasy [,/organic]",
                "detailed",
                "trending in artstation fantasy",
                "electric",
                "night",
            ]
            
            if enhance_checkbox.value:
                prompt = prompt + " " + " ".join(random.sample(enhancements, 5))
                print(f"Using enhanced prompt: {prompt}")    
            try:
                start_time = time.time()
                model.generate_images(
                    prompt,
                    num_images=num_images,
                    save_path="./output",
                )
                clear_output(wait=True)
                display_generated_images()
            except KeyboardInterrupt:
                print("\nUser interrupted image generation...")
            except Exception as e:
                print(f"An error occurred: {e}")
            finally:
                print(
                    f"Complete generating {num_images} images in './output' in {time.time() - start_time:.2f} seconds."
                )
                
    button.on_click(on_submit)

def display_generated_images(output_dir="output"):
    image_files = [f for f in os.listdir(output_dir) if f.endswith((".png", ".jpg"))]    
    num_images = len(image_files)
    grid_size = int(np.ceil(np.sqrt(num_images)))
    fig, axs = plt.subplots(grid_size, grid_size, figsize=(15, 15))    
    if num_images == 1:
        axs = np.array([[axs]])
    elif grid_size == 1:
        axs = np.array([axs])
    for ax, image_file in zip(axs.ravel(), image_files):
        img = mp_img.imread(os.path.join(output_dir, image_file))
        ax.imshow(img)
        ax.axis("off")  # Hide axes
    for ax in axs.ravel()[num_images:]:
        ax.axis("off") 
    plt.tight_layout()
    plt.show()


---

**Let's Dive In! And Experience the Art of Creation**

Eager to breathe life into your words? The interface below is your canvas. Set your preferences, type in your desired visual, and hit the "Generate Images!" button to witness the magic of Stable Diffusion.

What unfolds are unique images sculpted by the power of your words, each a manifestation of the expansive interpretations our model draws from your input.

This isn't just about AI; it's about the alchemy of your imagination combined with the prowess of Diffusion models. Every image stands as a symbol of the boundless potential of this synergy.

So, why wait? Let's paint the canvas with your imagination!

---

In [26]:
prompt_to_image()

Button(description='Generate Images!', style=ButtonStyle())

Output()

## Reference and Guidelines for Models Used in This Notebook

### CompVis/stable-diffusion-v1-4
- **Model card:** [CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4)
- **License:** CreativeML OpenRAIL M license
- **Reference:**
    ```bibtex
    @InProceedings{Rombach_2022_CVPR,
        author    = {Rombach, Robin and Blattmann, Andreas and Lorenz, Dominik and Esser, Patrick and Ommer, Bj\"orn},
        title     = {High-Resolution Image Synthesis With Latent Diffusion Models},
        booktitle = {Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR)},
        month     = {June},
        year      = {2022},
        pages     = {10684-10695}
    }
    ```

### stabilityai/stable-diffusion-2
- **Model card:** [stabilityai/stable-diffusion-2](https://huggingface.co/stabilityai/stable-diffusion-2)
- **License:** CreativeML Open RAIL++-M License
- **Reference:**
    ```bibtex
    @InProceedings{Rombach_2022_CVPR,
        author    = {Rombach, Robin and Blattmann, Andreas and Lorenz, Dominik and Esser, Patrick and Ommer, Bj\"orn},
        title     = {High-Resolution Image Synthesis With Latent Diffusion Models},
        booktitle = {Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR)},
        month     = {June},
        year      = {2022},
        pages     = {10684-10695}
    }
    ```

### stable-diffusion-xl-base-1.0
- **Model card:** [stabilityai/stable-diffusion-xl-base-1.0](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0)
- **License:** CreativeML Open RAIL++-M License


### Disclaimer for Using Stable Diffusion Models

The stable diffusion models provided here are powerful tools for high-resolution image synthesis, including text-to-image and image-to-image transformations. While they are designed to produce high-quality results, users should be aware of potential limitations:

- **Quality Variation:** The quality of generated images may vary based on the complexity of the input text or image, and the alignment with the model's training data.
- **Resource Consumption:** These models may require significant computational resources, and performance might vary depending on the hardware used.
- **Licensing and Usage Constraints:** Please carefully review the licensing information associated with each model to ensure compliance with all terms and conditions.
- **Ethical Considerations:** Consider the ethical implications of the generated content, especially in contexts that may involve sensitive or controversial subjects.

For detailed information on each model's capabilities, limitations, and best practices, please refer to the respective model cards and associated publications linked below.